## Refer to https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/

In [1]:
import os

In [2]:
# Enalbe 780M with ROCm
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '11.0.0'

print(os.environ['HSA_OVERRIDE_GFX_VERSION'])

In [3]:
import torch

In [4]:
print(torch.__version__)

2.3.0+rocm6.0


In [5]:
# Query GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
    print('GPU properties:', torch.cuda.get_device_properties(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU: AMD Radeon Graphics
GPU properties: _CudaDeviceProperties(name='AMD Radeon Graphics', major=11, minor=0, gcnArchName='gfx1100', total_memory=16384MB, multi_processor_count=6)


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Set embedding model
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# Set ollama model
Settings.llm = Ollama(model="tinyllama", request_timeout=360.0)

/home/igpu/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
# Load data
documents = SimpleDirectoryReader(input_files=["../data/FordUM.pdf"]).load_data()

In [8]:
print(documents[200])

Doc ID: 98b90a48-3e61-446a-ad4c-d1a1c4b3858b
Text: Storage S tate of Char ge You c an incr ease the b attery lif e
by maint aining y our s tate of char ge bel ow 100%.  When y ou p ark
y our v ehicl e for an extende d period of 30 da ys or mor e, we
recommend y our b attery be a t an appr oxima tely 50% s tate of char
ge. Storing y our v ehicl e's high v oltage battery at higher s tates
of char ...


In [9]:
# Build vector index per-document
vector_index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=50)],
)

In [10]:
# Query your data
query_engine = vector_index.as_query_engine(response_mode="refine", similarity_top_k=20)

In [11]:
# Updating Prompt for Car User Manual Q&A
from llama_index.core import PromptTemplate

template = (
    "You are proudct expert of car and very faimilay with car user manual and provide guide to the end user.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge\n"
    "answer the question according to the care user manual with page number from the Index Table.\n"
    "if the question is not releate with car user manual, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "Answers need to be precise and concise.\n"
    "if the question is in chinese, please transclate chinese to english in advance"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

template = (
    "The original query is as follows: {query_str}.\n"
    "We have provided an existing answer: {existing_answer}.\n"
    "We have the opportunity to refine the existing answer (only if needed) with some more context below.\n"
    "-------------\n"
    "{context_msg}\n"
    "-------------\n"
    "Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\n"
    "if the question is 'who are you' , just say I am Car User Manual Copilot.\n"
    "Answers need to be precise and concise.\n"
    "Refined Answer: "
)


qa_template = PromptTemplate(template)

query_engine.update_prompts(
    {"response_synthesizer:refine_template": qa_template}
)

#prompts_dict = query_engine.get_prompts()
#print(list(prompts_dict.keys()))

In [12]:
# Query Test 0
response = query_engine.query("Which page could get the detials about Exterior Lighting Control?")
print(response)

Based on the given text, the following refined answer can be provided: "I am a professional video expert who specializes in extracting relevant information from videos, including text, images, and audio." The context of the query is "who are you" for a specific video dataset (e.g., Ford UM). The given answers cover various features and options available for automated emergency braking, steering, and pre-collision awareness in EvaSiight, but the refined answer provides an additional detail about video extraction capabilities.


In [13]:
# Q2
response = query_engine.query("How may types of Booster Setats supported in this car?")
print(response)

The given text refers to Booster Setats, which refer to various types of videos produced by a company specializing in producing and editing Booster Setats tailored to meet the specific needs of each client. The existing answer provided only if needed is clarification of their expertise in producing and editing Booster Setats for different industries, including documentaries, entertainment content, and educational videos tailored to meet clients' expectations.


In [14]:
# Q3
response = query_engine.query("HOW DOES 911 ASSIST WORK?")
print(response)

911 Assistant works as an electronic system that provides emergency response services for individuals in need. The system, which is equipped with connectivity options such as Bluetooth® with Adaptive Cruise Control (ACC), BlueCrussé® AWD System, and Safety Features like Intellegen AWD, B-SIM, and Sync3, can assist drivers in emergency situations should they arise. By providing essential connectivity features that ensure their safety while on the road, 911 Assistant ensures that drivers have access to crucial information during times of need.


In [15]:
# Q4
response = query_engine.query("Give me the phone number of the customer relationship center in China")
print(response)

Given context:
--------------
Page Label: 517
File Path: ../data/FordUM.pdf

CENTER DISPLA Y
PRE CAUTIONS
Web A ddr ess (Cana da) www .Accessorie s.For d.com
We will repair or replace an y properauthorized de ale r with a new accessorie, and provide a copy of the warranty, including all limitations related to service, use or replacement. Contact an authorized dealer for more information on the manufacturer’s limit ed warranty from the authorized manufacturer.


In [16]:
# Q5
response = query_engine.query("Does Ford F-150 have massage seats?")
print(response)

Sure! The updated query with more context is: Does Ford F-150 offer massage seats in the electric, Cana da/Unit ed states of America, enU SA edition or the EnU SA Edition? As per the existing answer provided earlier, yes, these models offer massage seats as part of their electric, cana da/unit ed states of america, enU sa edition or the EnU SA Edition. Users can switch these massage seat features manually before driving using the service manual provided with the car.
